In [ ]:
%pip install torch supervision inference pillow google-genai

In [ ]:
import urllib.request

url = "https://raw.githubusercontent.com/jsoma/dataharvest25-ai-images-video/main/istockphoto-534232220-640_adpp_is.mp4"
urllib.request.urlretrieve(url, "istockphoto-534232220-640_adpp_is.mp4")

## Video processing: Car tracking

Want to reproduce [this Bloomberg piece about congestion pricing?](https://www.bloomberg.com/graphics/2025-nyc-congestion-pricing-week-one-traffic-mix-shifts/) We can get about... 60% of the way there!

This code is based heavily on [Piotr Skalski's presentation](https://x.com/i/broadcasts/1YqKDkdMOOYxV) at MIT from Nov 14, 2024. His is a good bit more interesting, though, as it talks about extracting license plates, OCRing them and tracking their movement in and out of a parking lot.

The library we're using for most of this is [supervision](https://supervision.roboflow.com/latest/), which is very very very tied to [Roboflow](https://roboflow.com/). Think of Roboflow as the the Hugging Face of vision models, except Hugging Face *does* have vision models and Roboflow's website is *not that easy to use*. Supervision is a fantastic library, though, so we're using it!

> If you want to use Hugging Face + pipelines + whatever models from over there, you just need to convert your detects into a supervision-friendly format. I just print out what my model gives me and ask an LLM.


## Our video

We're starting from a wonderful video from iStockPhoto because I don't have a bunch of videos of traffic laying around (sorry).

<div style="text-align: center"><video src="istockphoto-534232220-640_adpp_is.mp4" controls loop autoplay></div>

Let's see what we can do with it!

## Add detecting

In [ ]:
from IPython.display import clear_output, display
from PIL import Image
import os
import time
import supervision as sv
from inference import get_model

model = get_model("yolov10n-640")

video_path = "istockphoto-534232220-640_adpp_is.mp4"
frame_generator = sv.get_video_frames_generator(video_path)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

try:
    for frame in frame_generator:
        result = model.infer(frame)[0]
        detections = sv.Detections.from_inference(result)

        annotated_frame = frame.copy()
        
        annotated_frame = box_annotator.annotate(annotated_frame, detections)
    
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,
            detections=detections)
    
        # Display the image
        clear_output(wait=True)
        display(Image.fromarray(annotated_frame))
except KeyboardInterrupt:
    print("Exiting")

## Add tracking

In [ ]:
from IPython.display import clear_output, display
from PIL import Image
import os
import time
import supervision as sv
from inference import get_model

model = get_model("yolov10n-640")

video_path = "istockphoto-534232220-640_adpp_is.mp4"
frame_generator = sv.get_video_frames_generator(video_path)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

byte_track = sv.ByteTrack()
byte_track.reset()

trace_annotator = sv.TraceAnnotator()
smoother = sv.DetectionsSmoother()

try:
    for frame in frame_generator:
        result = model.infer(frame, confidence=0.3)[0]
        detections = sv.Detections.from_inference(result)
        detections = byte_track.update_with_detections(detections)
        detections = smoother.update_with_detections(detections)
        
        annotated_frame = frame.copy()
        labels = [
            f"{model.class_names[class_id]} {confidence:0.2f}"
            for _, _, confidence, class_id, tracker_id, _
            in detections
        ]
        
        annotated_frame = box_annotator.annotate(annotated_frame, detections)
        annotated_frame = trace_annotator.annotate(annotated_frame, detections)
    
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,
            detections=detections,
            labels=labels)
    
        # Display the image
        clear_output(wait=True)
        display(Image.fromarray(annotated_frame))
except KeyboardInterrupt:
    print("Exiting")

## Add counting

In [ ]:
from IPython.display import clear_output, display
from PIL import Image
import os
import time
import supervision as sv
from inference import get_model

model = get_model("yolov10n-640")

video_path = "istockphoto-534232220-640_adpp_is.mp4"
frame_generator = sv.get_video_frames_generator(video_path)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
line_zone_annotator = sv.LineZoneAnnotator(text_thickness=1)

byte_track = sv.ByteTrack()
byte_track.reset()

start = sv.Point(200, 175)
end = sv.Point(700, 175)
line_zone = sv.LineZone(start, end)
trace_annotator = sv.TraceAnnotator()
smoother = sv.DetectionsSmoother()

try:
    for frame in frame_generator:
        result = model.infer(frame, confidence=0.3)[0]
        detections = sv.Detections.from_inference(result)
        detections = byte_track.update_with_detections(detections)
        detections = smoother.update_with_detections(detections)
    
        line_zone.trigger(detections)
    
        annotated_frame = frame.copy()
        labels = [
            f"#{tracker_id} {model.class_names[class_id]} {confidence:0.2f}"
            for _, _, confidence, class_id, tracker_id, _
            in detections
        ]
        
        annotated_frame = box_annotator.annotate(annotated_frame, detections)
        annotated_frame = trace_annotator.annotate(annotated_frame, detections)
    
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,
            detections=detections,
            labels=labels)
    
        annotated_frame = line_zone_annotator.annotate(
            annotated_frame,
            line_counter=line_zone)
    
        # Display the image
        clear_output(wait=True)
        display(Image.fromarray(annotated_frame))
except KeyboardInterrupt:
    print("Exiting")

## Final version!

In [ ]:
from IPython.display import clear_output, display
from PIL import Image
import os
import time
import supervision as sv
from inference import get_model

model = get_model("yolov10n-640")

video_path = "istockphoto-534232220-640_adpp_is.mp4"
frame_generator = sv.get_video_frames_generator(video_path)

box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()
line_zone_annotator = sv.LineZoneAnnotator(text_thickness=1)

byte_track = sv.ByteTrack()
byte_track.reset()

start = sv.Point(200, 175)
end = sv.Point(700, 175)
line_zone = sv.LineZone(start, end)
trace_annotator = sv.TraceAnnotator()
smoother = sv.DetectionsSmoother()

try:
    for frame in frame_generator:
        result = model.infer(frame, confidence=0.3)[0]
        detections = sv.Detections.from_inference(result)
        detections = byte_track.update_with_detections(detections)
        detections = smoother.update_with_detections(detections)
    
        line_zone.trigger(detections)
    
        annotated_frame = frame.copy()
        labels = [
            f"#{tracker_id} {model.class_names[class_id]} {confidence:0.2f}"
            for _, _, confidence, class_id, tracker_id, _
            in detections
        ]
        
        annotated_frame = box_annotator.annotate(annotated_frame, detections)
        annotated_frame = trace_annotator.annotate(annotated_frame, detections)
    
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,
            detections=detections,
            labels=labels)
    
        annotated_frame = line_zone_annotator.annotate(
            annotated_frame,
            line_counter=line_zone)
    
        # Display the image
        clear_output(wait=True)
        display(Image.fromarray(annotated_frame))
except KeyboardInterrupt:
    print("Exiting")

## Going further!

What if you don't just want cars, though – you want to be *Bloomberg*, and spot taxis and box trucks and commercial vehicles and all sorts of specific cars! While you might try to find a model that does that for you, you can also **train your own** with a handful of pictures and videos.

If you're interested in going to the next level instead of just using out-of-the-box models, check out [the Roboflow documentation](https://blog.roboflow.com/getting-started-with-roboflow/).

# ...but also: can we do it with Gemini?

We're going to steal the code from [the video understanding notebook](01-Video%20understanding%20with%20Google%20Gemini.ipynb) and see how it does.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key='GEMINI API KEY GOES HERE')

In [ ]:
def upload_video(video_file_name):
  video_file = client.files.upload(file=video_file_name)

  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name)

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + video_file.uri)

  return video_file

In [ ]:
video = upload_video("istockphoto-534232220-640_adpp_is.mp4")

In [ ]:
from IPython.display import display, Markdown, HTML
from google.genai.types import GenerateContentConfig

prompt = """
Watch the traffic video. Provide a count of the number of cars that 
go into the left tunnel and the number of cars that come out of 
the right tunnel.
"""

video = video

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[
        video,
        prompt,
    ],
    config=GenerateContentConfig(temperature=0)
)

print(response.text)

**The actual number is around 20 and 20,** so no! Let's maybe not trust the all-knowing AI with this one.